<a href="https://colab.research.google.com/github/molan-zhang/urban-octo-train/blob/master/Copy_of_Encrypted_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Encrypted Deep Learning
---
Traing a Deep Learning Model
---
Let's build and train a toy deep learning model

In [0]:


#import necessary packets
from __future__ import print_function
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import sys

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# CIFAR 10 Test dataset and dataloader declaration
project_path = 'content/drive/My Drive/Colab Notebooks/Cybersecurity/NN Attacks/'
# Define a transform to normalize the data
transform = transforms.Compose([transforms.Resize(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# download CIFAR 10 training set
trainset = torchvision.datasets.CIFAR10(root= project_path+'/data', train=True,
                                        download=True, transform=transform)

# load the trainning set
trainloader = th.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

# download the test data
testset = torchvision.datasets.CIFAR10(root=project_path+'/data', train=False,
                                       download=True, transform=transform)

# load the test data
testloader = th.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# check those manually on the dataset site: https://www.cs.toronto.edu/~kriz/cifar.html 


Files already downloaded and verified
Files already downloaded and verified


In [0]:
# a toy model
criterion = nn.CrossEntropyLoss()

def train():
    # Training Logic
    opt = optim.Adam(params=model.parameters(), lr=0.001)
    running_loss = 0.0
    i = 0
    for data,target in trainloader:

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()
        i += 1

        # 2) make a prediction
        pred = model(data)
        targets = th.zeros([10])
        targets[target] = 1.0

        # 3) calculate how much we missed
        loss = ((pred - targets)**2).mean()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        running_loss += loss.item()
       
        # 6) print our progress
        if i % 100== 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  ( 1, i + 1, running_loss / 100))
            running_loss = 0.0


        

In [0]:
model = th.load("Alexnet_CIFAR_10.pkl")


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torchvision.models.alexnet.AlexNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `to

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(150528, 224)
        self.fc2 = nn.Linear(224, 10)

    def forward(self, x):
        x = x.reshape(150528)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [0]:
model = Net()

In [0]:
train()

In [0]:
data,labels = iter(testloader).next()
predictions = model(data)
print(predictions.data)
predictions = th.softmax(predictions.data, dim = 0)
print(predictions.data)

tensor([-0.1446, -0.0056, -0.1457, -0.0590, -0.0219,  0.0452, -0.1151, -0.0796,
        -0.0265,  0.0006])
tensor([0.0913, 0.1049, 0.0912, 0.0994, 0.1032, 0.1104, 0.0940, 0.0974, 0.1027,
        0.1055])


Encrypted Deep Learning using PySyft
---
Now let's use the library PySyft, which we introduced in the first section of this course, to cary out encrypted learning :


In [0]:
!pip install syft

In [0]:
import syft as sy
hook = sy.TorchHook(th)

In [0]:
# create a couple of workers
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# encrypt the model and share it among participants
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
for data,labels in trainloader :
  encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)
  encrypted_prediction = encrypted_model(encrypted_data)
  encrypted_prediction = encrypted_prediction.get().float_precision()
  encrypted_prediction = th.softmax(encrypted_prediction.data,dim = 1)
  final_pred = encrypted_prediction.max(0, keepdim=True)[1]
  print(final_pred)


tensor([1])
tensor([5])
tensor([4])


Additive Secrete Sharing
---


Additive Secret Sharing is a protocol for Multi-Party Computation. It allows multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x.
Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
x = 5
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted).

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share *= 2 
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. 
Decryption happens by summing all the shares together MODULUS some constant:

In [0]:
imgs,labels = iter(trainloader).next()
x = imgs[1]
print(x.shape)

Q = 23740 # large prime number

# bob_x_share = th.zeros([3,224,224])
bob_x_share = 23552 # <- a random number
alice_x_share = th.zeros([3,224,224])
alice_x_share = Q - bob_x_share + x 
print(x)
print(alice_x_share)

In [0]:
ans = th.zeros([3,224,224])
for i in range(3):
  for j in range(224):
    for k in range(224):
      x = (bob_x_share + alice_x_share[i][j][k]) % Q 
      if x < Q/2:
        ans[i][j][k] = x 
      else:
        ans[i][j][k] = x-Q
print(ans)
plt.imshow(ans.reshape([224,224,3]))

 Fixed Precision Encoding
 ---
 Additive secrete sharing works with integers. Thus, to apply this protocol on deep learning, we first need to convert the gradeints to integers. To do so, we used a predifned funciton fixed_precision. In the following, we illustrate its internat implemention in a very simple appraoch:

In [0]:
BASE=10
PRECISION=4

def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION
    

In [0]:
encode(0.25)

In [0]:
decode(2500)

---
Apply Addivite Secrete Sharing using PySyft on a real dataset (MNIST or CIFAR10)
---